## Imports

In [2]:
#imports
import pandas as pd
import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap

import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [3]:
# load the data
data_path = 'Datasets/2021-2023/CSB_2021-2023_All.csv'
initial_data = pd.read_csv(data_path)

/var/folders/n3/spvzfspj1gn51bldz75lhrt40000gn/T/ipykernel_73530/3892218257.py:3: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  initial_data = pd.read_csv(data_path)


In [4]:
# copy data (it's faster to copy than to load it again)
bike_data = initial_data.copy()

In [5]:
## Check for missing and Null data

In [6]:
# check for null values
bike_data.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    468710
start_station_id      468710
end_station_name      521058
end_station_id        521058
start_lat                  2
start_lng                  2
end_lat                13695
end_lng                13695
member_casual              0
dtype: int64

In [7]:
# check for missing values
bike_data.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    468710
start_station_id      468710
end_station_name      521058
end_station_id        521058
start_lat                  2
start_lng                  2
end_lat                13695
end_lng                13695
member_casual              0
dtype: int64

## Inspect the data

In [8]:
# check data types
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8180626 entries, 0 to 8180625
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 811.4+ MB


In [9]:
bike_data.describe()

,start_lat,start_lng,end_lat,end_lng
count,8.180624e+06,8.180624e+06,8.166931e+06,8.166931e+06
mean,3.890293e+01,-7.703172e+01,3.890184e+01,-7.703127e+01
std,2.656915e-02,3.286265e-02,5.567502e-02,1.025279e-01
min,3.877000e+01,-7.740000e+01,0.000000e+00,-7.756000e+01
25%,3.889050e+01,-7.704468e+01,3.889046e+01,-7.704468e+01
50%,3.890267e+01,-7.703174e+01,3.890206e+01,-7.703169e+01
75%,3.891369e+01,-7.701377e+01,3.891261e+01,-7.701350e+01
max,3.913000e+01,-7.682000e+01,3.918000e+01,0.000000e+00


In [10]:
# Because a lot of columns are of type object, we don't get much out of the data.
# Let's convert the column dtypes.

## Convert dtypes

In [11]:
# convert the object type columns to more useful data types

# convert categorical data
bike_data['rideable_type'] = pd.Categorical(bike_data['rideable_type'])
bike_data['member_casual'] = pd.Categorical(bike_data['member_casual'])

# convert datetime data
bike_data['started_at'] = pd.to_datetime(bike_data['started_at'])
bike_data['ended_at'] = pd.to_datetime(bike_data['ended_at'])

# droping the two rows with 'MTL-ECO5-03' as this is probably an error
bike_data = bike_data.drop(bike_data[bike_data['end_station_id'] == 'MTL-ECO5-03'].index)

# convert station id's to float
bike_data['end_station_id'] = pd.to_numeric(bike_data['end_station_id'], errors='coerce')
bike_data['start_station_id'] = pd.to_numeric(bike_data['start_station_id'], errors='coerce')

## Inspect the data again

In [12]:
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8180624 entries, 0 to 8180625
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       category      
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    float64       
 6   end_station_name    object        
 7   end_station_id      float64       
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       category      
dtypes: category(2), datetime64[ns](2), float64(6), object(3)
memory usage: 764.6+ MB


In [13]:
bike_data.describe()

,started_at,ended_at,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng
count,8180624,8180624,7.711914e+06,7.659566e+06,8.180624e+06,8.180624e+06,8.166931e+06,8.166931e+06
mean,2022-05-21 15:41:19.305605376,2022-05-21 16:05:15.981119744,3.136749e+04,3.136883e+04,3.890293e+01,-7.703172e+01,3.890184e+01,-7.703127e+01
min,2021-01-01 00:08:33,2021-01-01 00:21:00,3.020000e+04,3.020000e+04,3.877000e+01,-7.740000e+01,0.000000e+00,-7.756000e+01
25%,2021-10-05 22:01:12,2021-10-05 22:24:05.750000128,3.121200e+04,3.121400e+04,3.889050e+01,-7.704468e+01,3.889046e+01,-7.704468e+01
50%,2022-06-15 10:14:12.500000,2022-06-15 10:39:25,3.127700e+04,3.127500e+04,3.890267e+01,-7.703174e+01,3.890206e+01,-7.703169e+01
75%,2022-12-11 21:21:40.750000128,2022-12-11 21:35:58.500000,3.160500e+04,3.160800e+04,3.891369e+01,-7.701377e+01,3.891261e+01,-7.701350e+01
max,2023-06-30 23:59:35,2023-07-11 02:02:21,3.320000e+04,3.320000e+04,3.913000e+01,-7.682000e+01,3.918000e+01,0.000000e+00
std,NaN,NaN,2.585772e+02,2.581412e+02,2.656915e-02,3.286265e-02,5.567502e-02,1.025279e-01


In [14]:
# check for missing values
bike_data.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    468710
start_station_id      468710
end_station_name      521058
end_station_id        521058
start_lat                  0
start_lng                  0
end_lat                13693
end_lng                13693
member_casual              0
dtype: int64

In [15]:
# Now we see more in the data, but still not a lot based on the describe function.
# Interesting is that the last ride ended 11 days after the last ride started.
# We fortunately got rid of the missing data in start_lat/lng by deleting 'MTL-ECO5-03'.

In [50]:
bike_data

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8180621,E802A8D89245438B,electric_bike,2023-06-30 23:57:43,2023-07-01 00:07:16,NaN,NaN,14th & Newton St NW,31649.0,38.910000,-77.030000,38.931991,-77.032956,member
8180622,0448FEEB78F2B08D,classic_bike,2023-06-30 23:58:14,2023-07-01 00:24:14,18th & New Hampshire Ave NW,31324.0,Massachusetts Ave & Dupont Circle NW,31200.0,38.911268,-77.041829,38.910100,-77.044400,casual
8180623,5BE258BC59548BE9,classic_bike,2023-06-30 23:58:26,2023-07-01 00:20:06,18th & New Hampshire Ave NW,31324.0,Massachusetts Ave & Dupont Circle NW,31200.0,38.911268,-77.041829,38.910100,-77.044400,casual
8180624,5EAFC4B8F3E7680E,electric_bike,2023-06-30 23:59:01,2023-07-01 00:06:37,Adams Mill & Columbia Rd NW,31104.0,NaN,NaN,38.923005,-77.042420,38.930000,-77.030000,member


In [51]:
# get unique station names and ids
start_stations = bike_data[['start_station_name', 'start_station_id']].drop_duplicates(ignore_index=True)

# ignore NaN values
start_stations = start_stations[start_stations['start_station_name'].notna()]

#start_stations

In [52]:
# get unique station names and ids
end_stations = bike_data[['end_station_name', 'end_station_id']].drop_duplicates(ignore_index=True)

# ignore NaN values
end_stations = end_stations[end_stations['end_station_name'].notna()]

#end_stations

In [53]:
start_stations.rename(columns={
    'start_station_name' : 'station_name',
    'start_station_id' : 'station_id'
}, inplace=True)

#start_stations

In [54]:
end_stations.rename(columns={
    'end_station_name' : 'station_name',
    'end_station_id' : 'station_id'
}, inplace=True)

#end_stations

In [55]:
stations = pd.concat([start_stations, end_stations], ignore_index=True)
#stations

,station_name,station_id
0,Maine Ave & 9th St SW,31646.0
1,10th & U St NW,31111.0
2,17th & Corcoran St NW,31214.0
3,Wilson Blvd. & N. Vermont St.,31926.0
4,11th & Park Rd NW,31651.0
...,...,...
1632,Emma Lee St & Morris St,32266.0
1633,Route 29 & Bisvey Dr,32267.0
1634,Jones Branch Dr & Blyton St,32269.0
1635,Arlington Blvd & Allen St,32268.0


In [56]:
stations.drop_duplicates(subset='station_id', ignore_index=True, inplace=True)
stations

,station_name,station_id
0,Maine Ave & 9th St SW,31646.0
1,10th & U St NW,31111.0
2,17th & Corcoran St NW,31214.0
3,Wilson Blvd. & N. Vermont St.,31926.0
4,11th & Park Rd NW,31651.0
...,...,...
743,Arlington Blvd & Allen St,32268.0
744,Westbranch Dr & Jones Branch Dr,32270.0
745,Motivate Tech Office,32902.0
746,V1 Warehouse Test Station,32906.0


In [57]:
# calculate the number of rides starting from each station
station_counts = bike_data.groupby('start_station_id')['ride_id'].count().reset_index()
station_counts.rename(columns={'ride_id': 'start_station_count'}, inplace=True)

# merge the result with the 'stations' DataFrame
stations = pd.merge(stations, station_counts, left_on='station_id', right_on='start_station_id', how='left')
stations.drop(columns=['start_station_id'], inplace=True)

# replace NaN with 0 and convert to integers
stations['start_station_count'] = stations['start_station_count'].fillna(0).astype(int)


In [58]:
# calculate the number of rides ending at each station
station_counts = bike_data.groupby('end_station_id')['ride_id'].count().reset_index()
station_counts.rename(columns={'ride_id': 'end_station_count'}, inplace=True)

# merge the result with the 'stations' DataFrame
stations = pd.merge(stations, station_counts, left_on='station_id', right_on='end_station_id', how='left')
stations.drop(columns=['end_station_id'], inplace=True)

# replace NaN with 0 and convert to integers
stations['end_station_count'] = stations['end_station_count'].fillna(0).astype(int)

In [59]:
stations

,station_name,station_id,start_station_count,end_station_count
0,Maine Ave & 9th St SW,31646.0,40694,43141
1,10th & U St NW,31111.0,19801,19750
2,17th & Corcoran St NW,31214.0,51715,51910
3,Wilson Blvd. & N. Vermont St.,31926.0,3607,3430
4,11th & Park Rd NW,31651.0,24372,18892
...,...,...,...,...
743,Arlington Blvd & Allen St,32268.0,2,2
744,Westbranch Dr & Jones Branch Dr,32270.0,1,1
745,Motivate Tech Office,32902.0,0,12
746,V1 Warehouse Test Station,32906.0,0,8


In [60]:
# Are any of those stations maybe brand new and thus have only so little traffic?

In [62]:
# calculate the minimum started_at for each start station
started_at = bike_data.groupby('start_station_id')['started_at'].agg('min').reset_index()
started_at.rename(columns={'started_at': 'min_started_at'}, inplace=True)

# merge the result with the 'stations' DataFrame
stations = pd.merge(stations, started_at, left_on='station_id', right_on='start_station_id', how='left')
stations.drop(columns=['start_station_id'], inplace=True)

In [63]:
# calculate the maximum started_at for each start station
started_at = bike_data.groupby('start_station_id')['started_at'].agg('max').reset_index()
started_at.rename(columns={'started_at': 'max_started_at'}, inplace=True)

# merge the result with the 'stations' DataFrame
stations = pd.merge(stations, started_at, left_on='station_id', right_on='start_station_id', how='left')
stations.drop(columns=['start_station_id'], inplace=True)

In [66]:
# calculate how long the station has been online
stations['operation_time'] = stations['max_started_at'] - stations['min_started_at']

In [67]:
stations

,station_name,station_id,start_station_count,end_station_count,min_started_at,max_started_at,operation_time
0,Maine Ave & 9th St SW,31646.0,40694,43141,2021-01-01 00:08:33,2023-06-30 23:39:26,910 days 23:30:53
1,10th & U St NW,31111.0,19801,19750,2021-01-01 00:13:43,2023-06-30 23:17:45,910 days 23:04:02
2,17th & Corcoran St NW,31214.0,51715,51910,2021-01-01 00:14:32,2023-06-30 23:39:19,910 days 23:24:47
3,Wilson Blvd. & N. Vermont St.,31926.0,3607,3430,2021-01-01 00:15:45,2023-06-30 16:42:18,910 days 16:26:33
4,11th & Park Rd NW,31651.0,24372,18892,2021-01-01 00:17:46,2023-06-30 23:24:22,910 days 23:06:36
...,...,...,...,...,...,...,...
743,Arlington Blvd & Allen St,32268.0,2,2,2023-06-28 12:54:01,2023-06-29 17:09:48,1 days 04:15:47
744,Westbranch Dr & Jones Branch Dr,32270.0,1,1,2023-06-30 14:58:40,2023-06-30 14:58:40,0 days 00:00:00
745,Motivate Tech Office,32902.0,0,12,NaT,NaT,NaT
746,V1 Warehouse Test Station,32906.0,0,8,NaT,NaT,NaT


In [68]:
# Get location data to be able to put the stations on a map later

In [69]:
# get the lat for each station
end_station_lat = bike_data.groupby('end_station_id')['end_lat'].first().reset_index()

# merge the result with the 'stations' DataFrame
stations = pd.merge(stations, end_station_lat, left_on='station_id', right_on='end_station_id', how='left')
stations.drop(columns=['end_station_id'], inplace=True)

In [70]:
# get the lng for each station
end_station_lng = bike_data.groupby('end_station_id')['end_lng'].first().reset_index()

# merge the result with the 'stations' DataFrame
stations = pd.merge(stations, end_station_lng, left_on='station_id', right_on='end_station_id', how='left')
stations.drop(columns=['end_station_id'], inplace=True)

In [71]:
stations

,station_name,station_id,start_station_count,end_station_count,min_started_at,max_started_at,operation_time,end_lat,end_lng
0,Maine Ave & 9th St SW,31646.0,40694,43141,2021-01-01 00:08:33,2023-06-30 23:39:26,910 days 23:30:53,38.880714,-77.025327
1,10th & U St NW,31111.0,19801,19750,2021-01-01 00:13:43,2023-06-30 23:17:45,910 days 23:04:02,38.917200,-77.025900
2,17th & Corcoran St NW,31214.0,51715,51910,2021-01-01 00:14:32,2023-06-30 23:39:19,910 days 23:24:47,38.912100,-77.038700
3,Wilson Blvd. & N. Vermont St.,31926.0,3607,3430,2021-01-01 00:15:45,2023-06-30 16:42:18,910 days 16:26:33,38.879477,-77.114563
4,11th & Park Rd NW,31651.0,24372,18892,2021-01-01 00:17:46,2023-06-30 23:24:22,910 days 23:06:36,38.931322,-77.028247
...,...,...,...,...,...,...,...,...,...
743,Arlington Blvd & Allen St,32268.0,2,2,2023-06-28 12:54:01,2023-06-29 17:09:48,1 days 04:15:47,38.866315,-77.199799
744,Westbranch Dr & Jones Branch Dr,32270.0,1,1,2023-06-30 14:58:40,2023-06-30 14:58:40,0 days 00:00:00,38.931089,-77.220729
745,Motivate Tech Office,32902.0,0,12,NaT,NaT,NaT,0.000000,0.000000
746,V1 Warehouse Test Station,32906.0,0,8,NaT,NaT,NaT,38.963633,-77.010255


In [72]:
stations.isna().sum()

station_name           0
station_id             0
start_station_count    0
end_station_count      0
min_started_at         3
max_started_at         3
operation_time         3
end_lat                0
end_lng                0
dtype: int64

In [73]:
stations.describe()

,station_id,start_station_count,end_station_count,min_started_at,max_started_at,operation_time,end_lat,end_lng
count,748.000000,748.000000,748.000000,745,745,745,748.000000,748.000000
mean,31638.796791,10310.045455,10240.061497,2021-03-23 06:59:55.706040320,2023-06-25 00:37:14.610738176,823 days 17:37:18.904697984,38.859827,-76.961169
min,30200.000000,0.000000,1.000000,2021-01-01 00:08:33,2021-03-08 16:48:30,0 days 00:00:00,0.000000,-77.368375
25%,31246.750000,865.250000,849.500000,2021-01-01 10:53:05,2023-06-30 14:14:42,901 days 23:41:34,38.878834,-77.092046
50%,31628.500000,3427.000000,3265.000000,2021-01-02 11:26:30,2023-06-30 20:09:25,909 days 07:35:31,38.900472,-77.043485
75%,32011.250000,15014.750000,15130.500000,2021-01-06 16:22:26,2023-06-30 22:58:28,910 days 10:02:52,38.936099,-77.009644
max,33200.000000,82520.000000,81486.000000,2023-06-30 14:58:40,2023-06-30 23:59:35,910 days 23:30:53,39.125828,0.000000
std,449.498833,14364.921426,14557.629633,NaN,NaN,211 days 18:03:55.046132464,1.424126,2.819231


In [132]:
# take into account only stations operating for at least 180 days
half_year = pd.Timedelta(days=180)
established_stations = stations[(stations['operation_time'] > half_year)].copy()

In [133]:
# calculate rides per day (starting and ending)
established_stations['rides/day'] = (established_stations['start_station_count'] + established_stations['end_station_count']) / established_stations['operation_time'].dt.days

In [134]:
established_stations['low_use'] = established_stations['rides/day'] < 4
established_stations

,station_name,station_id,start_station_count,end_station_count,min_started_at,max_started_at,operation_time,end_lat,end_lng,rides/day,low_use
0,Maine Ave & 9th St SW,31646.0,40694,43141,2021-01-01 00:08:33,2023-06-30 23:39:26,910 days 23:30:53,38.880714,-77.025327,92.126374,False
1,10th & U St NW,31111.0,19801,19750,2021-01-01 00:13:43,2023-06-30 23:17:45,910 days 23:04:02,38.917200,-77.025900,43.462637,False
2,17th & Corcoran St NW,31214.0,51715,51910,2021-01-01 00:14:32,2023-06-30 23:39:19,910 days 23:24:47,38.912100,-77.038700,113.873626,False
3,Wilson Blvd. & N. Vermont St.,31926.0,3607,3430,2021-01-01 00:15:45,2023-06-30 16:42:18,910 days 16:26:33,38.879477,-77.114563,7.732967,False
4,11th & Park Rd NW,31651.0,24372,18892,2021-01-01 00:17:46,2023-06-30 23:24:22,910 days 23:06:36,38.931322,-77.028247,47.542857,False
...,...,...,...,...,...,...,...,...,...,...,...
706,Marymount University / Yorktown Blvd & 26th St N,31977.0,194,166,2022-10-21 15:55:20,2023-06-28 10:12:43,249 days 18:17:23,38.904252,-77.127776,1.445783,True
707,Reston Town Center Metro South,32258.0,137,129,2022-10-28 13:27:04,2023-06-29 19:21:31,244 days 05:54:27,38.952300,-77.361466,1.090164,True
708,Fort Lincoln Dr & 33rd Pl NE,31537.0,268,246,2022-12-13 19:37:49,2023-06-30 14:05:37,198 days 18:27:48,38.921641,-76.955687,2.595960,True
709,South Dakota Ave & Bladensburg Rd NE,31538.0,209,219,2022-12-17 10:30:17,2023-06-30 22:56:33,195 days 12:26:16,38.925200,-76.964896,2.194872,True


In [135]:
established_stations['low_use'].value_counts()

low_use
False    476
True     234
Name: count, dtype: int64

In [136]:
#half_year = pd.Timedelta(days=180)
#low_use_stations = stations[(stations['operation_time'] > half_year) & (stations['start_station_count'] < 360)]
#low_use_stations

In [137]:
established_stations[established_stations['low_use']].describe()

,station_id,start_station_count,end_station_count,min_started_at,max_started_at,operation_time,end_lat,end_lng,rides/day
count,234.000000,234.000000,234.000000,234,234,234,234.000000,234.000000,234.000000
mean,32051.512821,709.371795,682.747863,2021-03-16 12:16:50.564102400,2023-06-24 01:31:38.029914624,829 days 13:14:47.465811968,38.940512,-77.110773,1.679894
min,31054.000000,91.000000,74.000000,2021-01-01 03:45:59,2021-08-20 00:08:41,195 days 04:57:02,38.782633,-77.368375,0.209821
25%,31934.250000,328.000000,312.500000,2021-01-02 14:16:28.500000,2023-06-28 19:31:06.750000128,848 days 16:38:58.500000,38.869526,-77.194249,0.859627
50%,32061.500000,579.000000,584.500000,2021-01-06 09:33:36,2023-06-30 09:52:15,903 days 10:32:30,38.931210,-77.106627,1.444257
75%,32227.750000,985.750000,962.000000,2021-02-09 16:30:37.750000128,2023-06-30 17:39:41.750000128,908 days 18:20:11.500000,38.998874,-76.993827,2.454518
max,32901.000000,1989.000000,1716.000000,2022-12-17 10:33:36,2023-06-30 23:52:01,910 days 17:02:52,39.125828,-76.825535,3.997800
std,286.375818,464.581507,445.018060,NaN,NaN,156 days 05:35:31.464600836,0.089179,0.135089,1.014434


In [138]:
established_stations[established_stations['low_use']].sort_values(by='end_station_count', ascending=False)

,station_name,station_id,start_station_count,end_station_count,min_started_at,max_started_at,operation_time,end_lat,end_lng,rides/day,low_use
78,Benning Rd & East Capitol St NE / Benning Rd M...,31707.0,1685,1716,2021-01-01 04:07:26,2023-06-30 07:37:59,910 days 03:30:33,38.889935,-76.937230,3.737363,True
605,Tysons Corner Station,32204.0,1734,1713,2021-01-20 13:43:23,2023-06-30 19:18:08,891 days 05:34:45,38.920939,-77.222115,3.868687,True
330,Pennsylvania & Minnesota Ave SE,31805.0,1460,1690,2021-01-02 09:59:45,2023-06-30 14:27:08,909 days 04:27:23,38.873057,-76.971015,3.465347,True
291,Oglethorpe St & 42nd Ave,32408.0,1545,1684,2021-01-02 05:35:53,2023-06-30 17:20:15,909 days 11:44:22,38.959370,-76.946138,3.552255,True
80,Minnesota Ave & R St SE,31826.0,1694,1656,2021-01-01 04:47:56,2023-06-30 09:09:18,910 days 04:21:22,38.869337,-76.978519,3.681319,True
...,...,...,...,...,...,...,...,...,...,...,...
607,Ridge Rd Community Center,31718.0,131,114,2021-01-25 23:20:48,2023-06-13 12:32:01,868 days 13:11:13,38.876227,-76.940981,0.282258,True
710,South Dakota Ave & 33rd St NE,31539.0,91,106,2022-12-17 10:33:36,2023-06-30 15:30:38,195 days 04:57:02,38.921986,-76.960696,1.010256,True
486,Medical Center Dr & Key West Ave,32041.0,101,87,2021-01-02 15:38:05,2023-06-18 09:42:30,896 days 18:04:25,39.103091,-77.196442,0.209821,True
639,Inlet Ct & Wiehle Dr,32242.0,121,82,2021-07-18 09:03:49,2023-06-26 06:10:24,707 days 21:06:35,38.966006,-77.334170,0.287129,True


In [139]:
# show the low use stations on the map
m=folium.Map(location=[38.90, -77.06], zoom_start=12)

# Add markers to the map from the DataFrame
for index, row in low_use_stations.iterrows():
    folium.Marker([row['end_lat'], row['end_lng']],
                  popup=row['station_name']).add_to(m)

m

In [141]:
m=folium.Map(location=[38.90, -77.06], zoom_start=12)
hexbin_data = list(zip(low_use_stations['end_lat'], low_use_stations['end_lng']))
HeatMap(hexbin_data, radius=15, blur=5).add_to(m)

m